# Webscraping 2.0

Joseph Nelson, DC

In today's codealong, I'll walkthrough how to build a scraper using urllib and BeautifulSoup. We'll discover the problems we discussed in the lesson readme associated with doing so, and we'll remedy this problem using a headless browser called Selenium.

For starter's we're going to be scraping OpenTable's DC listings. We're interested in knowing the restaurant's **name, location, price, and how many people booked it today.**

OpenTable provides all of this information on this given page: http://www.opentable.com/washington-dc-restaurant-listings

Let's inspect the elements of this page to assure we can find each of the bits of information in which we're interested.
>Class proceeds to do exactly that ^

We'll then build our scraper:

In [ ]:
# import our necessary first packages
from bs4 import BeautifulSoup
import urllib

In [ ]:
# set the url we want to visit
url = "http://www.opentable.com/washington-dc-restaurant-listings"

# visit that url, and grab the html of said page
html = urllib.urlopen(url).read()

At this point, what is in html?

In [ ]:
html

In [ ]:
# we need to convert this into a soup object
soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")

Let's first find each restaurant name listed on the page we've loaded. How do we find the page location of the restaurant? (Psst: we need to know where in the **html** the restaurant element is housed)

See if you can find the restaurant name on the page. Keep in mind there are many restaurants loaded on the page.

In [ ]:
# print the restaurant names


Now that we can find each element, let's think how we can loop through them all one-by-one. In the following cell, print out the name (and **only** the clean name, not the rest of the html) of each restaurant.

In [ ]:
# for each element you find, print out the restaurant name


Great!

Can you repeat that process for finding the location? 

In [ ]:
# first, see if you can identify the location for all elements -- print it out


In [ ]:
# now print out EACH location for the restaurants


Ok, we've figured out the restaurant name and location. Now we need to grab the price (number of dollar signs on a scale of one to four) for each restaurant. We'll follow the same process.

In [ ]:
# print out all prices


In [ ]:
# print out EACH number of dollar signs per restaurant
# this one is trickier to eliminate the html. Hint: try a nested find


That looks great, but what if I wanted just the number of dollar signs per restaurant? Can you figure out a way to simply print out the number of dollar signs per restaurant listed?

In [ ]:
# print the number of dollars signs per restaurant


Phew, nice work. 

One more, right? We only need to find the number times a restaurant was booked. In the next cell, print out all objects that contain the number of times the restaurant was booked.

In [ ]:
# print out all objects that contain the number of times the restaurant was booked


That's weird -- an empty set. Did we find the wrong element? What's going on here? Discuss.

How can we debug this? Any ideas?

In [ ]:
# let's first try printing out all 'span' class objects
for entry in soup.find_all('span'):
    print entry

I still don't see it. Let's search our entire soup object:

In [ ]:
# print out soup, do command+f for "booked "
soup

What do you notice? Why is this happening?

## Enter Selenium

Selenium is a headless browser. That means it enables us to mock human browsing behavior -- even waiting for JavaScript elements to load.

If you do not already have Selenium installed, you can do so via pip. Simply: `pip install selenium`

In [ ]:
# import
from selenium import webdriver

Selenium requires us to determine a default browser to run. I'm going to opt for Firefox, but Chromium is also a very common choice. http://selenium-python.readthedocs.io/faq.html

In [ ]:
# STOP
# what is going to happen when I run the next cell?

In [ ]:
# create a driver
driver = webdriver.Chrome(executable_path="/Users/mjspeck/Downloads/chromedriver")

Pretty crazy, right? Let's close that driver.

In [ ]:
# close it
driver.close()

In [ ]:
# let's boot it up, and visit a URL of our choice
driver = webdriver.Chrome(executable_path="/Users/mjspeck/Downloads/chromedriver")
driver.get("http://www.python.org")

Awesome. Now we're getting somewhere: programmatically controlling our browser like a human.

In [ ]:
driver = webdriver.Chrome(executable_path="/Users/mjspeck/Downloads/chromedriver")
driver.get("http://www.python.org")

Let's return to our problem at hand. We need to visit the OpenTable listing for DC. Once there, we need to get the html to load. In the next cell, prove you can programmatically visit the page.

In [ ]:
# visit our OpenTable page
driver = webdriver.Chrome(executable_path="/Users/mjspeck/Downloads/chromedriver")
driver.get("http://www.opentable.com/washington-dc-restaurant-listings")
# always good to check we've got the page we think we do
assert "OpenTable" in driver.title

Now, to resolve our JavaScript problem, there's a few things we can do. What I'll do in this case is request that the page load, wait one second, and then I'm going to grab the source html from the page. Because the page should believe I'm visiting from a live connection on a browser client, the JavaScript should render to be a part of the page source. I can then grab the page source.

In [ ]:
# import sleep
from time import sleep

In [ ]:
# visit our relevant page
driver = webdriver.Chrome(executable_path="/Users/mjspeck/Downloads/chromedriver")
driver.get("http://www.opentable.com/washington-dc-restaurant-listings")
# wait one second
sleep(1)
#grab the page source
html = driver.page_source

Pop quiz: what do we need to do with this html?

In [ ]:
# BeautifulSoup it!
html = BeautifulSoup(html)

Now, let's return to our earlier problem: how do we locate bookings on the page?

In [ ]:
# print out the number bookings for all restaurants
print html.find_all('div', {'class':'booking'})

In [ ]:
# now print out each booking for the listings using a loop
for booking in html.find_all('div', {'class':'booking'}):
    print booking

Let's grab just the text of each of these entries.

In [ ]:
# do the same as above, but grabbing only the text content
for booking in html.find_all('div', {'class':'booking'}):
    print booking.text

We've succeeded!

But we can clean this up a little bit. We're going to use regular expressions (regex) to grab only the digits that are available in each of the text.

The best way to get good at regex is to, well, just keep trying and testing: http://pythex.org/

In [ ]:
# import regex
import re

Given we haven't covered regex, I'll show you how to use the search function to match any given digit.

In [ ]:
# for each entry, grab the text
for booking in html.find_all('div', {'class':'booking'}):
    # match all digits
    match = re.search(r'\d+', booking.text)
    # print if found
    if match:
        print match.group()
    # otherwise pass
    else:
        pass

Before we demonstrate all the other amazing things about headless browsers, let's finish up collecting the data we want from this current example. Do you suppose the html parsing we wrote above will still work on the page source we've grabbed from our headless browser?

To be most efficient, we want to only do a single loop for each entry on the page. That means we want to find what element all of other other elements (name, location, price, bookings) is housed within. Where on the page is each entry located?

In [ ]:
# print out all entries
print html.find_all('div', {'class':'result content-section-list-row cf with-times'})

Look over the page. Does every single entry have each element we're seeking?
> I did this previously. I know for a fact that not every element has a number of recent bookings. That's probably exactly why OpenTable houses this in JavaScript: they want to continously update the number of bookings with the most relevant number of values.

In [ ]:
# what happens when a booking is not available?
# print out each booking entry, using the identification code we wrote above
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    print entry.find('div', {'class':'booking'})

In [ ]:
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    print entry.find('div', {'class':'booking'}).text

What do you notice takes the place when booking is not found?

Thus, we will use exceptions. Here's a demo:

In [ ]:
# if we find the element we want, we print it. Otherwise, we print 'ZERO'
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    try:
        print entry.find('div', {'class':'booking'}).text
    except:
        print 'ZERO'

From previously completing this, I know all other elements WILL be returned. That means we do not have to create exceptions for them.

However, the onus is on you to now put all the pieces together.

Loop through each entry. For each entry, grab the relevant information we want (name, location, price, bookings). Produce a dataframe with the columns "name","location","price","bookings" that contains the 100 entries we would like.

In [ ]:
# I'm going to create my empty df first
import pandas as pd
dc_eats = pd.DataFrame(columns=["name","location","price","bookings"])

**Check:** What is my for-loop doing?

In [ ]:
# loop through each entry
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    # grab the name
    name = entry.find('span', {'class': 'rest-row-name-text'}).text
    # grab the location
    location = entry.find('span', {'class': 'rest-row-meta--location rest-row-meta-text'}).renderContents()
    # grab the price
    price = entry.find('div', {'class': 'rest-row-pricing'}).find('i').renderContents().count('$')
    # try to find the number of bookings
    try:
        temp = entry.find('div', {'class':'booking'}).text
        match = re.search(r'\d+', temp)
        if match:
            bookings = match.group()
    except:
        bookings = 'NA'
    # add to df
    dc_eats.loc[len(dc_eats)]=[name, location, price, bookings]

In [ ]:
# check out our work
dc_eats.head()

Awesome! We succeeded.

Now, let's explore some of the other functionality of a webdriver. We've barely scratched the surface.

In [ ]:
# we can send keys as well
# import
from selenium.webdriver.common.keys import Keys

In [ ]:
# open Chrome
driver = webdriver.Chrome(executable_path="/Users/mjspeck/Downloads/chromedriver")

In [ ]:
# visit Python
driver.get("http://www.python.org")
# verify we're in the right place
assert "Python" in driver.title

In [ ]:
# find the search position
elem = driver.find_element_by_name("q")
# clear it
elem.clear()
# type in pycon
elem.send_keys("pycon")


In [ ]:
# send those keys
elem.send_keys(Keys.RETURN)
# no results
assert "No results found." not in driver.page_source

In [ ]:
# close
driver.close()

In [ ]:
# all at once:
driver = webdriver.Chrome(executable_path="/Users/mjspeck/Downloads/chromedriver")
driver.get("http://www.python.org")
assert "Python" in driver.title
elem = driver.find_element_by_name("q")
elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)
assert "No results found." not in driver.page_source
driver.close()

The above example (and many others) are available in the Selenium docs: http://selenium-python.readthedocs.io/getting-started.html

What is especially important is exploring functionality like locating elements: http://selenium-python.readthedocs.io/locating-elements.html#locating-elements

FAQ:
http://selenium-python.readthedocs.io/faq.html